In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Longevity_and_Aging/GSE48264'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Uppsala Longitudinal Study of Adult Men (ULSAM)"
!Series_summary	"The Uppsala Longitudinal Study of Adult Men is a population-based study aimed at identifying risk factors for cardiovascular disease. At the time of biopsy all subjects were ~ 70yr of age"
!Series_overall_design	"We extracted RNA from muscle tissue taken from 129 subjects, when they were aged ~70yr and characterised as disease-free (note the above average longevity in Swedes born circa 1920 compared with US and UK populations). From these samples, 108 yielded RNA of sufficient quality to profile on Affymetrix gene-chips."
!Series_overall_design	"Only survival data are used in the paper."
!Series_overall_design	"There are no data from cardiovascular disease subjects; we only profiled the healthy subjects and followed for 20yrs."
Sample Characteristics Dictionary:
{0: ['disease-free: disease-free'], 1: ['age(approx): 70 yr'], 2: ['tissue: skeletal muscle biopsy (baseline)'], 3: ['surv

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
# Given that the data is obtained from RNA profiled on Affymetrix gene-chips, it is likely to contain gene expression data.
is_gene_available = True 

# 2. Variable Availability and Data Type Conversion
# Given the available keys, let's identify the relevant rows:

trait_row = 3  # Longevity_and_Aging ('survival') is available at key 3.
age_row = 1  # age is explicitly recorded under key 1, all values are approximately 70.
gender_row = None  # gender information is not explicitly mentioned or inferred from the summary.

def convert_trait(value):
    # Extract the actual value from format like 'survival: None'
    term = value.split(': ')[1]
    if term == 'None':
        return 0
    elif term in ['Hosp', 'Death']:
        return 1
    else:
        return None

def convert_age(value):
    # Extract the actual value from format like 'age(approx): 70 yr'
    term = value.split(': ')[1]
    try:
        return float(term.split(' ')[0])
    except:
        return None

# For gender_row being None, we do not need to define `convert_gender`.
convert_gender = None

# 3. Save Metadata
import os
json_path = './preprocessed/Longevity_and_Aging/cohort_info.json'
os.makedirs(os.path.dirname(json_path), exist_ok=True)
save_cohort_info('GSE48264', json_path, is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    csv_path = './preprocessed/Longevity_and_Aging/trait_data/GSE48264.csv'
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Longevity_and_Aging', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1173505': [0.0, 70.0], 'GSM1173506': [1.0, 70.0], 'GSM1173507': [1.0, 70.0], 'GSM1173508': [1.0, 70.0], 'GSM1173509': [1.0, 70.0], 'GSM1173510': [1.0, 70.0], 'GSM1173511': [0.0, 70.0], 'GSM1173512': [1.0, 70.0], 'GSM1173513': [1.0, 70.0], 'GSM1173514': [1.0, 70.0], 'GSM1173515': [1.0, 70.0], 'GSM1173516': [1.0, 70.0], 'GSM1173517': [1.0, 70.0], 'GSM1173518': [0.0, 70.0], 'GSM1173519': [1.0, 70.0], 'GSM1173520': [1.0, 70.0], 'GSM1173521': [0.0, 70.0], 'GSM1173522': [1.0, 70.0], 'GSM1173523': [0.0, 70.0], 'GSM1173524': [1.0, 70.0], 'GSM1173525': [1.0, 70.0], 'GSM1173526': [1.0, 70.0], 'GSM1173527': [1.0, 70.0], 'GSM1173528': [1.0, 70.0], 'GSM1173529': [0.0, 70.0], 'GSM1173530': [1.0, 70.0], 'GSM1173531': [1.0, 70.0], 'GSM1173532': [0.0, 70.0], 'GSM1173533': [1.0, 70.0], 'GSM1173534': [1.0, 70.0], 'GSM1173535': [1.0, 70.0], 'GSM1173536': [1.0, 70.0], 'GSM1173537': [0.0, 70.0], 'GSM1173538': [1.0, 70.0], 'GSM1173539': [1.0, 70.0], 'GSM1173540': [1.0, 70.0], 'GSM1173541': [0.0, 70.0], 

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['2315251', '2315373', '2315554', '2315633', '2315674', '2315739',
       '2315894', '2315918', '2315951', '2316069', '2316218', '2316245',
       '2316379', '2316558', '2316605', '2316746', '2316905', '2316953',
       '2317246', '2317317'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Map keys for identifiers and gene symbols in gene annotations
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes 
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and obtain the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping)
